In [ ]:
import logging
import threading
from queue import Queue
import time
import json
import pandas as pd
from collections import Counter
from typing import List

In [ ]:
logger = logging.getLogger('multithreading-example')
logger.setLevel(logging.INFO)
sh = logging.StreamHandler()
sh.setFormatter(logging.Formatter( '%(asctime)s - %(name)s - %(levelname)s - %(message)s'))
logger.addHandler(sh)

In [ ]:
q = Queue()
input_list_to_process = list(range(100))
for input_id in input_list_to_process:
    q.put(input_id)

In [ ]:
%%time
# The actual work done for a task = processing one list in memory
import gc
def do_work(input_id):
    try:
        foo = 1
        logger.info('Did step 1 for %s',input_id)
        bar = 2
        logger.info('Did step 2 for %s',input_id)
        res = foo + bar + input_id
        logger.info('FINISHED %s',input_id)
        logger.info('-'*50)
        return res
    except:
        print(f'An exception occured while processing {input_id}')

# Each thread indefinitely keeps reading from the queue and doing a task
def threader():
    while True:#not q.empty():
        # get the job from the front of the queue
        do_work(q.get())
        q.task_done()
    
# To control the number of active threads of tasks
num_threads = 4
for i in range(num_threads):
    thread = threading.Thread(target = threader )
    # this ensures the thread will die when the main thread dies
    # can set t.daemon to False if you want it to keep running
    thread.daemon = True
    thread.start()

# blocks the main thread until the workers have processed everything that's in the queue
q.join()
print('All work completed')